<a href="https://colab.research.google.com/github/Trantracy/Analyse-the-relation-between-customers-and-Supplier/blob/master/Advanced_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://i.imgur.com/0AUxkXt.png)

# MLE - Advanced SQL

### Connect to the database

![](https://raw.githubusercontent.com/jpwhite3/northwind-SQLite3/master/Northwind_ERD.png)

**Run those cells below**

In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
import sqlite3
import pandas as pd

# Replace the path below with yours
conn = sqlite3.connect('gdrive/My Drive/Student Files/FTMLE - Tonga/Data/Northwind_small.sqlite')

In [0]:
# Show all tables in the database
pd.read_sql_query("SELECT name FROM sqlite_master WHERE type = 'table'", conn)

,name
0,Employee
1,Category
2,Customer
3,Shipper
4,Supplier
5,Order
6,Product
7,OrderDetail
8,CustomerCustomerDemo
9,CustomerDemographic


## Exercises

### 1. Query all employees in the same department as Janet
Note: Your query cannot contain "Sales Representative"

__Expected Output:__
![Expected Answer](https://i.imgur.com/QbMyadp.png)

In [0]:
# Query all employees in the same department as Janet
# Note: Your query cannot contain "Sales Representative"

query = '''
          SELECT * 
          FROM Employee
          WHERE Title = (
            SELECT 
              Title
            FROM Employee
            WHERE FirstName = "Janet");
'''

pd.read_sql_query(query, conn)

### 2a. Without using DISTINCT, query countries that have customers but no suppliers
__Expected Output:__

![Expected Answer](https://imgur.com/sdUKDYz.png)


In [0]:
# Without using DISTINCT, query countries that have customers but no suppliers
query = '''
          SELECT Country
          FROM Customer
          EXCEPT
          SELECT Country
          FROM Supplier
'''

pd.read_sql_query(query, conn)

### 2b. Similar to Question 2a but now use DISTINCT

In [0]:
# Without using DISTINCT, query countries that have customers but no suppliers
query = '''
          SELECT DISTINCT Country
          FROM Customer
          WHERE Country NOT IN (
            SELECT Country
            FROM Supplier
          )
'''

pd.read_sql_query(query, conn)

,Country
0,Mexico
1,Argentina
2,Switzerland
3,Austria
4,Portugal
5,Venezuela
6,Ireland
7,Belgium
8,Poland


### 3. Query top 5 most potential customers in North America and top 5 most potential customers in South America
Note: Potential customers are evaluated based on the total revenue earned.

__Expected Output__

![Expected Answer](https://i.imgur.com/ctNWmXu.png)

In [0]:
# Query top 5 most potential customers in North America and top 5 most potential customers in South America
query = '''
            SELECT * 
            FROM (
              SELECT 
                CompanyName, 
                Region, 
                SUM((UnitPrice - Discount) * Quantity) AS Total
              FROM "Customer" AS c
                JOIN "Order" AS o 
                  ON c.Id = o.CustomerId
                JOIN "OrderDetail" AS d 
                  ON o.Id = d.OrderId
              WHERE Region = "North America"
              GROUP BY c.Id
              ORDER BY Total DESC
              LIMIT 10
            ) 
            UNION
            SELECT *
            FROM (
              SELECT 
                CompanyName, 
                Region, 
                SUM((UnitPrice - Discount) * Quantity) AS Total
              FROM "Customer" AS c
                JOIN "Order" AS o 
                  ON c.Id = o.CustomerId
                JOIN "OrderDetail" AS d 
                  ON o.Id = d.OrderId
              WHERE Region = "South America"
              GROUP BY c.Id
              ORDER BY Total DESC
              LIMIT 10
              ) ORDER BY Region, Total DESC;
'''

pd.read_sql_query(query, conn)

In [0]:
# Query top 5 most potential customers in North America and top 5 most potential customers in South America
query = '''
            WITH orders AS (SELECT * FROM 'order')

            SELECT* FROM
            (SELECT 
              CompanyName, 
              Region, 
              SUM(UnitPrice*Quantity) AS Total 
            FROM orders 
              JOIN customer
                ON orders.customerid = customer.id 
              JOIN orderdetail 
                ON orders.id = orderdetail.orderid 
            WHERE Region LIKE 'north %' 
            GROUP BY CompanyName 
            ORDER BY Total DESC 
            LIMIT 10)
         
            UNION
            SELECT * FROM(
            SELECT 
              CompanyName, 
              Region, 
              SUM(UnitPrice * Quantity) AS Total 
            FROM orders 
              JOIN customer 
                ON orders.customerid = customer.id 
              JOIN orderdetail 
                ON orders.id = orderdetail.orderid 
            WHERE Region LIKE 'north %' 
            GROUP BY CompanyName
            ORDER BY Total DESC 
            LIMIT 10)
'''

pd.read_sql_query(query, conn)

,CompanyName,Region,Total
0,Bottom-Dollar Markets,North America,22607.70
1,Great Lakes Food Market,North America,19711.13
2,Let's Stop N Shop,North America,3490.02
3,Lonesome Pine Restaurant,North America,4258.60
4,Mère Paillarde,North America,32203.90
5,Old World Delicatessen,North America,16325.15
6,Rattlesnake Canyon Grocery,North America,52245.90
7,Save-a-lot Markets,North America,115673.39
8,Split Rail Beer & Ale,North America,12489.70
9,White Clover Markets,North America,29073.45


### 4. Query the number of products in each Popularity group
__Rules:__
- Products with more than 400 items are "Regular" products
- Products with more than 800 items sold are "Popular" products
- Product with less than 400 items sold are "Unpopular" products

__Expected Output__

![](https://i.imgur.com/OhmmmAt.png)

In [0]:
# Query the number of products in each Popularity group

query = '''
          SELECT 
            Popularity, 
            COUNT(Popularity) AS "Count"
          FROM (
            SELECT (
              CASE
                WHEN Quantity > 800 
                  THEN "Popular"
                WHEN Quantity > 400 
                  THEN "Regular"
                ELSE "Unpopular"
              END) AS Popularity
            FROM (
              SELECT 
                p.ProductName, 
                SUM(d.Quantity) AS Quantity
              FROM OrderDetail AS d
              JOIN Product AS p
                ON d.ProductId = p.Id
              GROUP BY p.ProductName
              )
            )
            GROUP BY Popularity

'''

pd.read_sql_query(query, conn)

,Popularity,Count
0,Popular,23
1,Regular,35
2,Unpopular,19


### 5. Query the contact list of Customer and Supplier by City

In [0]:
# Query the contact list of Customers and Suppliers by City
# Sort the City column alphabetically

query = '''
         SELECT 
          City, 
          CompanyName, 
          ContactName, 
          'Customer' as Relationship 
         FROM Customer
         UNION
         SELECT 
          City, 
          CompanyName, 
          ContactName, 
          'Supplier'
         FROM Supplier
         ORDER BY City, CompanyName;

'''

pd.read_sql_query(query, conn)


,City,CompanyName,ContactName,Relationship
0,Aachen,Drachenblut Delikatessen,Sven Ottlieb,Customers
1,Albuquerque,Rattlesnake Canyon Grocery,Paula Wilson,Customers
2,Anchorage,Old World Delicatessen,Rene Phillips,Customers
3,Ann Arbor,Grandma Kelly's Homestead,Regina Murphy,Suppliers
4,Annecy,Gai pâturage,Eliane Noz,Suppliers
...,...,...,...,...
115,Versailles,La corne d'abondance,Daniel Tonini,Customers
116,Walla Walla,Lazy K Kountry Store,John Steel,Customers
117,Warszawa,Wolski Zajazd,Zbyszek Piestrzeniewicz,Customers
118,Zaandam,Zaanse Snoepfabriek,Dirk Luchte,Suppliers


In [0]:


  # Query the contact list of Customers and Suppliers by City
# Sort the City column alphabetically

query = '''
       SELECT * 
        FROM 
        (SELECT CompanyName,Region,Amount as Total
        FROM Customer AS c
          JOIN order_revenue  AS odr
          ON odr.customerId = c.id
        GROUP BY c.id
        HAVING c.Region Like 'North America'
        ORDER BY Total DESC)

'''

pd.read_sql_query(query, conn)


DatabaseError: ignored